In [151]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [152]:
data, meta = scipy.io.arff.loadarff('../mtr-datasets/atp7d.arff')
data = pd.DataFrame(data)
data.head()

,"""'departwb'""","""'departdaytype'""","""'fetchdays'""","""'fetchdow'""","""'dtd'""","""'ALLminpA'""","""'ALLminpA'l1""","""'ALLminpA'l2""","""'ALLminpA'l7""","""'ALLavgpA'""",...,"""fdisoweekday4""","""fdisoweekday5""","""fdisoweekday6""","""fdisoweekday7""",LBL+ALLminpA+bt7d_000,LBL+ALLminp0+bt7d_000,LBL+aDLminpA+bt7d_000,LBL+aCOminpA+bt7d_000,LBL+aFLminpA+bt7d_000,LBL+aUAminpA+bt7d_000
0,72.0,1.0,60.0,3.0,12.0,359.0,372.0,351.0,317.0,615.96,...,0.0,0.0,0.0,0.0,326.0,675.0,626.0,545.0,326.0,545.0
1,79.0,1.0,69.0,5.0,10.0,336.0,298.0,298.0,320.0,616.73,...,0.0,1.0,0.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
2,79.0,1.0,70.0,6.0,9.0,336.0,336.0,298.0,320.0,628.87,...,0.0,0.0,1.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
3,79.0,1.0,71.0,7.0,8.0,336.0,336.0,336.0,330.0,632.73,...,0.0,0.0,0.0,1.0,315.0,675.0,631.0,587.0,315.0,587.0
4,79.0,1.0,72.0,1.0,7.0,336.0,336.0,336.0,330.0,635.47,...,0.0,0.0,0.0,0.0,315.0,1182.0,720.0,720.0,315.0,720.0


In [153]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=100)

In [154]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (207, 411)
Train_Y:  (207, 6)
Test_X:  (89, 411)
Test_Y:  (89, 6)


In [155]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))


## Bidirectional Model


In [156]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
#from keras.backend.tensorflow_backend import clear_session
from sklearn.model_selection import train_test_split

In [157]:
ENS_COUNT = 6
#clear_session()

In [158]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='glorot_uniform'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

## Forward Ensemble

In [159]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_forward = []

    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(y_t.shape[1]):
        print("Training chain node ",i)
        y_now = y_t[:,[i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_forward.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_fwd.append(chain_forward)

TRAINING ENSEMBLE 0 :



Training chain node  0
Shapes:
 X = (207, 411) 
 Y = (207, 1)
Epoch 1/500
5/5 [==============================] - 0s 1ms/step - loss: 1.4293 - mse: 1.4293
Epoch 2/500
5/5 [==============================] - 0s 1ms/step - loss: 0.7877 - mse: 0.7877
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4304 - mse: 0.4304
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.2370 - mse: 0.2370
Epoch 5/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1844 - mse: 0.1844
Epoch 6/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1522 - mse: 0.1522
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1133 - mse: 0.1133
Epoch 8/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0893 - mse: 0.0893
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0740 - mse: 0.0740
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0933 - mse:

5/5 [==============================] - 0s 1ms/step - loss: 0.0500 - mse: 0.0500
Epoch 88/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0478 - mse: 0.0478
Epoch 89/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0317 - mse: 0.0317
Epoch 90/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0407 - mse: 0.0407
Epoch 91/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0170 - mse: 0.0170
Epoch 92/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0176 - mse: 0.0176
Epoch 93/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0135 - mse: 0.0135
Epoch 94/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0080 - mse: 0.0080
Epoch 95/500
5/5 [==============================] - ETA: 0s - loss: 0.0157 - mse: 0.015 - 0s 1ms/step - loss: 0.0106 - mse: 0.0106
Epoch 96/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102
Epoch 97/500
5/5 [===========================

5/5 [==============================] - 0s 1ms/step - loss: 0.0298 - mse: 0.0298
Epoch 172/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0140 - mse: 0.0140
Epoch 173/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0136 - mse: 0.0136
Epoch 174/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0113 - mse: 0.0113
Epoch 175/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0074 - mse: 0.0074
Epoch 176/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0132 - mse: 0.0132
Epoch 177/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0080 - mse: 0.0080
Epoch 178/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0066 - mse: 0.0066
Epoch 179/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0045 - mse: 0.0045
Epoch 180/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0032 - mse: 0.0032
Epoch 181/500
5/5 [==============================] - 0s 1ms/step - loss: 0

Epoch 256/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0058 - mse: 0.0058
Epoch 257/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0035 - mse: 0.0035
Epoch 258/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0032 - mse: 0.0032
Epoch 259/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0033 - mse: 0.0033
Epoch 260/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0028 - mse: 0.0028
Epoch 261/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025
Epoch 262/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018
Epoch 263/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0010 - mse: 0.0010
Epoch 264/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 265/500
5/5 [==============================] - 0s 1ms/step - loss: 7.8480e-04 - mse: 7.8480e-04
Epoch 266/500
5/5 [==============================] -

5/5 [==============================] - 0s 1ms/step - loss: 0.0015 - mse: 0.0015    
Epoch 343/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0018 - mse: 0.0018
Epoch 344/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 345/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011
Epoch 346/500
5/5 [==============================] - 0s 1ms/step - loss: 9.0755e-04 - mse: 9.0755e-04
Epoch 347/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0028 - mse: 0.0028    
Epoch 348/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 349/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0037 - mse: 0.0037
Epoch 350/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0026 - mse: 0.0026
Epoch 351/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0030 - mse: 0.0030
Epoch 352/500
5/5 [==============================] - 0s 2m

...

5/5 [==============================] - 0s 1ms/step - loss: 0.0020 - mse: 0.0020
Epoch 91/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0027 - mse: 0.0027
Epoch 92/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0036 - mse: 0.0036
Epoch 93/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0024 - mse: 0.0024
Epoch 94/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0085 - mse: 0.0085
Epoch 95/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0068 - mse: 0.0068
Epoch 96/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0031 - mse: 0.0031
Epoch 97/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0037 - mse: 0.0037
Epoch 98/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049
Epoch 99/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0027 - mse: 0.0027
Epoch 100/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0026 - m

5/5 [==============================] - 0s 1ms/step - loss: 0.0052 - mse: 0.0052
Epoch 177/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050
Epoch 178/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0032 - mse: 0.0032
Epoch 179/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0029 - mse: 0.0029
Epoch 180/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0042 - mse: 0.0042
Epoch 181/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0051 - mse: 0.0051
Epoch 182/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0041 - mse: 0.0041
Epoch 183/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0042 - mse: 0.0042
Epoch 184/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0037 - mse: 0.0037
Epoch 185/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022
Epoch 186/500
5/5 [==============================] - 0s 2ms/step - loss: 0

Epoch 263/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0183 - mse: 0.0183
Epoch 264/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0241 - mse: 0.0241
Epoch 265/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0203 - mse: 0.0203
Epoch 266/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0176 - mse: 0.0176
Epoch 267/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0348 - mse: 0.0348
Epoch 268/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0171 - mse: 0.0171
Epoch 269/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0204 - mse: 0.0204
Epoch 270/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0158 - mse: 0.0158
Epoch 271/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0231 - mse: 0.0231
Epoch 272/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0147 - mse: 0.0147
Epoch 273/500
5/5 [==============================] - 0s 1ms/

5/5 [==============================] - 0s 1ms/step - loss: 2.4024e-07 - mse: 2.4024e-07
Epoch 346/500
5/5 [==============================] - 0s 1ms/step - loss: 2.7113e-07 - mse: 2.7113e-07
Epoch 347/500
5/5 [==============================] - 0s 1ms/step - loss: 3.1148e-07 - mse: 3.1148e-07
Epoch 348/500
5/5 [==============================] - 0s 1ms/step - loss: 2.5141e-07 - mse: 2.5141e-07
Epoch 349/500
5/5 [==============================] - 0s 1ms/step - loss: 3.0375e-07 - mse: 3.0375e-07
Epoch 350/500
5/5 [==============================] - 0s 2ms/step - loss: 4.4884e-07 - mse: 4.4884e-07
Epoch 351/500
5/5 [==============================] - 0s 1ms/step - loss: 3.1267e-07 - mse: 3.1267e-07
Epoch 352/500
5/5 [==============================] - 0s 2ms/step - loss: 2.9218e-07 - mse: 2.9218e-07
Epoch 353/500
5/5 [==============================] - 0s 1ms/step - loss: 2.3599e-07 - mse: 2.3599e-07
Epoch 354/500
5/5 [==============================] - 0s 1ms/step - loss: 1.9923e-07 - mse: 1.992

5/5 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 426/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0023 - mse: 0.0023
Epoch 427/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0080 - mse: 0.0080    
Epoch 428/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0183 - mse: 0.0183
Epoch 429/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0287 - mse: 0.0287
Epoch 430/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0174 - mse: 0.0174
Epoch 431/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0620 - mse: 0.0620
Epoch 432/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0472 - mse: 0.0472
Epoch 433/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0350 - mse: 0.0350
Epoch 434/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0215 - mse: 0.0215
Epoch 435/500
5/5 [==============================] - 0s 1ms/step - los

## Backward  Ensemble

In [160]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_reverse = []
    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print("Training chain node ",i)
        y_now = y_t[:,[-i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_reverse.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_rev.append(chain_reverse)

TRAINING ENSEMBLE 0 :



Training chain node  1
Shapes:
 X = (207, 411) 
 Y = (207, 1)
Epoch 1/500
5/5 [==============================] - 0s 1ms/step - loss: 0.7849 - mse: 0.7849
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.4015 - mse: 0.4015
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.2707 - mse: 0.2707
Epoch 4/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1912 - mse: 0.1912
Epoch 5/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1509 - mse: 0.1509
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0984 - mse: 0.0984
Epoch 7/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0967 - mse: 0.0967
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0943 - mse: 0.0943
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0831 - mse: 0.0831
Epoch 10/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0868 - mse:

5/5 [==============================] - 0s 1ms/step - loss: 0.0133 - mse: 0.0133
Epoch 87/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103
Epoch 88/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081
Epoch 89/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0068 - mse: 0.0068
Epoch 90/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0062 - mse: 0.0062
Epoch 91/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0142 - mse: 0.0142
Epoch 92/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101
Epoch 93/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0084 - mse: 0.0084
Epoch 94/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0128 - mse: 0.0128
Epoch 95/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105
Epoch 96/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0081 - ms

5/5 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 171/500
5/5 [==============================] - 0s 1ms/step - loss: 9.5665e-04 - mse: 9.5665e-04
Epoch 172/500
5/5 [==============================] - 0s 1ms/step - loss: 9.7063e-04 - mse: 9.7063e-04
Epoch 173/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0010 - mse: 0.0010
Epoch 174/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 175/500
5/5 [==============================] - 0s 2ms/step - loss: 8.8755e-04 - mse: 8.8755e-04
Epoch 176/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0018 - mse: 0.0018    
Epoch 177/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022    
Epoch 178/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0017 - mse: 0.0017
Epoch 179/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0033 - mse: 0.0033
Epoch 180/500
5/5 [===============

5/5 [==============================] - 0s 1ms/step - loss: 5.1244e-04 - mse: 5.1244e-04
Epoch 257/500
5/5 [==============================] - 0s 2ms/step - loss: 2.9975e-04 - mse: 2.9975e-04
Epoch 258/500
5/5 [==============================] - 0s 2ms/step - loss: 3.8767e-04 - mse: 3.8767e-04
Epoch 259/500
5/5 [==============================] - 0s 2ms/step - loss: 2.5565e-04 - mse: 2.5565e-04
Epoch 260/500
5/5 [==============================] - 0s 2ms/step - loss: 2.8472e-04 - mse: 2.8472e-04
Epoch 261/500
5/5 [==============================] - 0s 2ms/step - loss: 2.5551e-04 - mse: 2.5551e-04
Epoch 262/500
5/5 [==============================] - 0s 2ms/step - loss: 1.7984e-04 - mse: 1.7984e-04
Epoch 263/500
5/5 [==============================] - 0s 2ms/step - loss: 2.7888e-04 - mse: 2.7888e-04
Epoch 264/500
5/5 [==============================] - 0s 2ms/step - loss: 2.5830e-04 - mse: 2.5830e-04
Epoch 265/500
5/5 [==============================] - 0s 2ms/step - loss: 1.8638e-04 - mse: 1.863

5/5 [==============================] - 0s 1ms/step - loss: 0.1046 - mse: 0.1046
Epoch 341/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1578 - mse: 0.1578
Epoch 342/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1304 - mse: 0.1304
Epoch 343/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2055 - mse: 0.2055
Epoch 344/500
5/5 [==============================] - 0s 1ms/step - loss: 0.2201 - mse: 0.2201
Epoch 345/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1772 - mse: 0.1772
Epoch 346/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1690 - mse: 0.1690
Epoch 347/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0594 - mse: 0.0594
Epoch 348/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0752 - mse: 0.0752
Epoch 349/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0463 - mse: 0.0463
Epoch 350/500
5/5 [==============================] - 0s 1ms/step - loss: 0

...

5/5 [==============================] - 0s 2ms/step - loss: 3.8249e-04 - mse: 3.8249e-04
Epoch 148/500
5/5 [==============================] - 0s 2ms/step - loss: 2.9467e-04 - mse: 2.9467e-04
Epoch 149/500
5/5 [==============================] - 0s 1ms/step - loss: 3.3582e-04 - mse: 3.3582e-04
Epoch 150/500
5/5 [==============================] - 0s 1ms/step - loss: 3.1884e-04 - mse: 3.1884e-04
Epoch 151/500
5/5 [==============================] - 0s 2ms/step - loss: 4.8101e-04 - mse: 4.8101e-04
Epoch 152/500
5/5 [==============================] - 0s 2ms/step - loss: 3.8154e-04 - mse: 3.8154e-04
Epoch 153/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0019 - mse: 0.0019    
Epoch 154/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018
Epoch 155/500
5/5 [==============================] - 0s 1ms/step - loss: 9.0050e-04 - mse: 9.0050e-04
Epoch 156/500
5/5 [==============================] - 0s 2ms/step - loss: 5.4132e-04 - mse: 5.4132e-04
Epoch 

5/5 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 232/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 233/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025
Epoch 234/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019
Epoch 235/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0024 - mse: 0.0024
Epoch 236/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0028 - mse: 0.0028
Epoch 237/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0063 - mse: 0.0063
Epoch 238/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094
Epoch 239/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0069 - mse: 0.0069
Epoch 240/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0080 - mse: 0.0080
Epoch 241/500
5/5 [==============================] - 0s 1ms/step - loss: 0

5/5 [==============================] - 0s 1ms/step - loss: 0.0033 - mse: 0.0033    
Epoch 315/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0022 - mse: 0.0022    
Epoch 316/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 317/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 318/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016    
Epoch 319/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0015 - mse: 0.0015
Epoch 320/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0010 - mse: 0.0010
Epoch 321/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013    
Epoch 322/500
5/5 [==============================] - 0s 2ms/step - loss: 9.5289e-04 - mse: 9.5289e-04
Epoch 323/500
5/5 [==============================] - 0s 2ms/step - loss: 9.9267e-04 - mse: 9.9267e-04
Epoch 324/500
5/5 [===============

5/5 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 399/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 400/500
5/5 [==============================] - 0s 1ms/step - loss: 7.2752e-04 - mse: 7.2752e-04
Epoch 401/500
5/5 [==============================] - 0s 2ms/step - loss: 7.4648e-04 - mse: 7.4648e-04
Epoch 402/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012    
Epoch 403/500
5/5 [==============================] - 0s 2ms/step - loss: 8.5322e-04 - mse: 8.5322e-04
Epoch 404/500
5/5 [==============================] - 0s 2ms/step - loss: 7.7183e-04 - mse: 7.7183e-04
Epoch 405/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0020 - mse: 0.0020    
Epoch 406/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025
Epoch 407/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0379 - mse: 0.0379
Epoch 408/500
5/5 [===========

Epoch 481/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0010 - mse: 0.0010
Epoch 482/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 483/500
5/5 [==============================] - 0s 1ms/step - loss: 8.1983e-04 - mse: 8.1983e-04
Epoch 484/500
5/5 [==============================] - 0s 2ms/step - loss: 6.4158e-04 - mse: 6.4158e-04
Epoch 485/500
5/5 [==============================] - 0s 2ms/step - loss: 5.3982e-04 - mse: 5.3982e-04
Epoch 486/500
5/5 [==============================] - 0s 1ms/step - loss: 5.3075e-04 - mse: 5.3075e-04
Epoch 487/500
5/5 [==============================] - 0s 2ms/step - loss: 3.9760e-04 - mse: 3.9760e-04
Epoch 488/500
5/5 [==============================] - 0s 2ms/step - loss: 4.6322e-04 - mse: 4.6322e-04
Epoch 489/500
5/5 [==============================] - 0s 2ms/step - loss: 1.6242e-04 - mse: 1.6242e-04
Epoch 490/500
5/5 [==============================] - 0s 1ms/step - loss: 1.8936e-04 - mse: 1.8936e

## Perceptron fuzzifier

## Training The Fuzzifier

In [161]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_train.copy()
    X_feed_re = X_train.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (207, 411)
Getting op from chain node  1
Shapes:
 X = (207, 412)
Getting op from chain node  2
Shapes:
 X = (207, 413)
Getting op from chain node  3
Shapes:
 X = (207, 414)
Getting op from chain node  4
Shapes:
 X = (207, 415)
Getting op from chain node  5
Shapes:
 X = (207, 416)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (207, 411)
Getting op from chain node  1
Shapes:
 X = (207, 412)
Getting op from chain node  2
Shapes:
 X = (207, 413)
Getting op from chain node  3
Shapes:
 X = (207, 414)
Getting op from chain node  4
Shapes:
 X = (207, 415)
Getting op from chain node  5
Shapes:
 X = (207, 416)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (207, 411)
Getting op from chain node  1
Shapes:
 X = (207, 412)
Getting op from chain node  2
Shapes:
 X = (207, 413)
Getting op from chain node  3
Shapes:
 X = (207, 414)
Getting op from chain node  4
Shapes:
 X = (207, 415)
Getting op from chain node  5
Shap

In [162]:
## Merged Ensemble Reverse
forward_merged = np.array(y_full_fw).reshape(np.array(y_full_fw).shape[0]*np.array(y_full_fw).shape[1],
                           np.array(y_full_fw).shape[2])

## Merged Ensemble Reverse
reverse_merged = np.array(y_full_re).reshape(np.array(y_full_re).shape[0]*np.array(y_full_re).shape[1],
                           np.array(y_full_re).shape[2])

In [163]:
forward_merged[:,0]

array([0.73832303, 2.5289962 , 2.1729262 , 1.5665473 , 0.33188045,
       1.5179237 , 0.7372341 , 2.5184321 , 2.1738882 , 1.7757726 ,
       0.25696737, 1.5121412 , 0.7577766 , 3.137057  , 2.2687619 ,
       1.6084034 , 0.31069025, 1.5160244 ], dtype=float32)

In [164]:
full_set = np.append(forward_merged,reverse_merged,axis=0).T
full_set.shape

(207, 36)

In [165]:
## Creating FCNN Fuzzifier

In [166]:
fuzzifier = Sequential()
fuzzifier.add(Dense(y_train.shape[1],activation='linear',input_dim = full_set.shape[1]))
fuzzifier.compile(optimizer='adam',loss='mse')
fuzzifier.summary()

Model: "sequential_129"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_338 (Dense)            (None, 6)                 222       
Total params: 222
Trainable params: 222
Non-trainable params: 0
_________________________________________________________________


In [167]:
#Training Fuzzifier
fuzzifier.fit(full_set,y_train,epochs=500,batch_size=5)

Epoch 1/500
42/42 [==============================] - 0s 1ms/step - loss: 1.7002
Epoch 2/500
42/42 [==============================] - 0s 1ms/step - loss: 0.8640
Epoch 3/500
42/42 [==============================] - 0s 1ms/step - loss: 0.5369
Epoch 4/500
42/42 [==============================] - 0s 1ms/step - loss: 0.4004
Epoch 5/500
42/42 [==============================] - 0s 1ms/step - loss: 0.3220
Epoch 6/500
42/42 [==============================] - 0s 1ms/step - loss: 0.2671
Epoch 7/500
42/42 [==============================] - 0s 1ms/step - loss: 0.2328
Epoch 8/500
42/42 [==============================] - 0s 1ms/step - loss: 0.1991
Epoch 9/500
42/42 [==============================] - 0s 1ms/step - loss: 0.1736
Epoch 10/500
42/42 [==============================] - 0s 1ms/step - loss: 0.1545
Epoch 11/500
42/42 [==============================] - 0s 1ms/step - loss: 0.1351
Epoch 12/500
42/42 [==============================] - 0s 1ms/step - loss: 0.1187
Epoch 13/500
42/42 [=================

42/42 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 103/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 104/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 105/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 106/500
42/42 [==============================] - 0s 999us/step - loss: 0.0019
Epoch 107/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 108/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0018
Epoch 109/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 110/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 111/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0019
Epoch 112/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0017
Epoch 113/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0018
Epoch 114/500
42/42 [=======

42/42 [==============================] - 0s 1ms/step - loss: 0.0013
Epoch 202/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0017
Epoch 203/500
42/42 [==============================] - 0s 1ms/step - loss: 9.8566e-04
Epoch 204/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 205/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 206/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 207/500
42/42 [==============================] - 0s 1ms/step - loss: 9.1876e-04
Epoch 208/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 209/500
42/42 [==============================] - 0s 1ms/step - loss: 7.8296e-04
Epoch 210/500
42/42 [==============================] - 0s 1ms/step - loss: 8.6975e-04
Epoch 211/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 212/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 213/500


42/42 [==============================] - 0s 1ms/step - loss: 6.3321e-04
Epoch 299/500
42/42 [==============================] - 0s 1ms/step - loss: 6.4280e-04
Epoch 300/500
42/42 [==============================] - 0s 1ms/step - loss: 6.7463e-04
Epoch 301/500
42/42 [==============================] - 0s 1ms/step - loss: 6.9468e-04
Epoch 302/500
42/42 [==============================] - 0s 1ms/step - loss: 8.3057e-04
Epoch 303/500
42/42 [==============================] - 0s 1ms/step - loss: 6.5850e-04
Epoch 304/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 305/500
42/42 [==============================] - 0s 1ms/step - loss: 7.6280e-04
Epoch 306/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0018
Epoch 307/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 308/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 309/500
42/42 [==============================] - 0s 1ms/step - loss: 9.4258e-0

42/42 [==============================] - 0s 1ms/step - loss: 5.0131e-04
Epoch 395/500
42/42 [==============================] - 0s 1ms/step - loss: 6.2269e-04
Epoch 396/500
42/42 [==============================] - 0s 1ms/step - loss: 6.1911e-04
Epoch 397/500
42/42 [==============================] - 0s 1ms/step - loss: 8.4379e-04
Epoch 398/500
42/42 [==============================] - 0s 1ms/step - loss: 5.4296e-04
Epoch 399/500
42/42 [==============================] - 0s 1ms/step - loss: 7.8921e-04
Epoch 400/500
42/42 [==============================] - 0s 1ms/step - loss: 8.3345e-04
Epoch 401/500
42/42 [==============================] - 0s 1ms/step - loss: 7.5875e-04
Epoch 402/500
42/42 [==============================] - 0s 1ms/step - loss: 8.2786e-04
Epoch 403/500
42/42 [==============================] - 0s 1ms/step - loss: 7.6648e-04
Epoch 404/500
42/42 [==============================] - 0s 1ms/step - loss: 6.6372e-04
Epoch 405/500
42/42 [==============================] - 0s 1ms/step -

42/42 [==============================] - 0s 1ms/step - loss: 3.3262e-04
Epoch 490/500
42/42 [==============================] - 0s 1ms/step - loss: 4.6766e-04
Epoch 491/500
42/42 [==============================] - 0s 1ms/step - loss: 4.4666e-04
Epoch 492/500
42/42 [==============================] - 0s 1ms/step - loss: 3.9486e-04
Epoch 493/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 494/500
42/42 [==============================] - 0s 1ms/step - loss: 7.7495e-04
Epoch 495/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 496/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0020
Epoch 497/500
42/42 [==============================] - 0s 1ms/step - loss: 0.0018
Epoch 498/500
42/42 [==============================] - 0s 998us/step - loss: 0.0019
Epoch 499/500
42/42 [==============================] - 0s 1ms/step - loss: 6.2227e-04
Epoch 500/500
42/42 [==============================] - 0s 1ms/step - loss: 3.0171e-04


### Testing FCNN Fuzzifier

In [168]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
        
## Merged Ensemble Reverse
forward_merged = np.array(y_full_fw).reshape(np.array(y_full_fw).shape[0]*np.array(y_full_fw).shape[1],
                           np.array(y_full_fw).shape[2])

## Merged Ensemble Reverse
reverse_merged = np.array(y_full_re).reshape(np.array(y_full_re).shape[0]*np.array(y_full_re).shape[1],
                           np.array(y_full_re).shape[2])

full_set_test = np.append(forward_merged,reverse_merged,axis=0).T
print("\n\nTest shape:, ",full_set_test.shape)



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416

In [169]:
predictions = fuzzifier.predict(full_set_test)
print(predictions.shape)

(89, 6)


In [170]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))

RMSE:  45.688489015408685
MAE:  28.451226080848034
R_Squared:  0.7129504236654673


## Convolutional Fuzzifier

In [171]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_train.copy()
    X_feed_re = X_train.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
y_full_fw = np.array(y_full_fw)[:,:,:,0]
y_full_re = np.array(y_full_re)[:,:,:,0]




ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (207, 411)
Getting op from chain node  1
Shapes:
 X = (207, 412)
Getting op from chain node  2
Shapes:
 X = (207, 413)
Getting op from chain node  3
Shapes:
 X = (207, 414)
Getting op from chain node  4
Shapes:
 X = (207, 415)
Getting op from chain node  5
Shapes:
 X = (207, 416)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (207, 411)
Getting op from chain node  1
Shapes:
 X = (207, 412)
Getting op from chain node  2
Shapes:
 X = (207, 413)
Getting op from chain node  3
Shapes:
 X = (207, 414)
Getting op from chain node  4
Shapes:
 X = (207, 415)
Getting op from chain node  5
Shapes:
 X = (207, 416)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (207, 411)
Getting op from chain node  1
Shapes:
 X = (207, 412)
Getting op from chain node  2
Shapes:
 X = (207, 413)
Getting op from chain node  3
Shapes:
 X = (207, 414)
Getting op from chain node  4
Shapes:
 X = (207, 415)
Getting op from chain node  5
Shap

In [172]:
intervened = []
for forward,reverse in zip(y_full_fw,y_full_re):
    forward = forward.T
    reverse = reverse.T
    data = []
    for i in range(forward.shape[0]):
        row = []
        flipped_reversed = list(reverse[i])[::-1]
        for j in range(forward.shape[1]):
            row.append(forward[i,j])
            row.append(flipped_reversed[j])
        
        data.append(row)
    intervened.append(data)
intervened = np.array(intervened)

In [173]:
full_set = intervened[0,:,:]
for i in range(1,np.array(intervened).shape[0]):
    full_set = np.append(full_set,intervened[i,:,:],axis = 1)

In [174]:
from keras.layers import Conv1D,Flatten
#CNN Fuzzifier 
fuzzifier = Sequential()
fuzzifier.add(Conv1D(filters=1,kernel_size=2,strides=2,activation='relu',input_shape=(full_set.shape[1],1)))
fuzzifier.add(Flatten())
fuzzifier.add(Dense(128,activation='relu'))
fuzzifier.add(Dense(y_train.shape[1],activation='linear'))
fuzzifier.compile(optimizer='adam',loss='mse')

In [175]:
fuzzifier.summary()

Model: "sequential_130"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 18, 1)             3         
_________________________________________________________________
flatten_5 (Flatten)          (None, 18)                0         
_________________________________________________________________
dense_339 (Dense)            (None, 128)               2432      
_________________________________________________________________
dense_340 (Dense)            (None, 6)                 774       
Total params: 3,209
Trainable params: 3,209
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Training Fuzzifier
fuzzifier.fit(full_set.reshape(full_set.shape[0],full_set.shape[1],1),y_train,epochs=500,batch_size=5)

### Testing CNN Fuzzifier

In [177]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
y_full_fw = np.array(y_full_fw)[:,:,:,0]
y_full_re = np.array(y_full_re)[:,:,:,0]




ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416

In [178]:
intervened = []
for forward,reverse in zip(y_full_fw,y_full_re):
    forward = forward.T
    reverse = reverse.T
    data = []
    for i in range(forward.shape[0]):
        row = []
        flipped_reversed = list(reverse[i])[::-1]
        for j in range(forward.shape[1]):
            row.append(forward[i,j])
            row.append(flipped_reversed[j])
        
        data.append(row)
    intervened.append(data)
intervened = np.array(intervened)

In [179]:
full_set_test = intervened[0,:,:]
for i in range(1,np.array(intervened).shape[0]):
    full_set_test = np.append(full_set_test,intervened[i,:,:],axis = 1)

In [180]:
full_set_test = full_set_test.reshape(full_set_test.shape[0],full_set_test.shape[1],1)
predictions = fuzzifier.predict(full_set_test)
print(predictions.shape)

(89, 6)


In [181]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))

RMSE:  46.44949223387919
MAE:  26.50196812661846
R_Squared:  0.6613315255623619


## Mean Aggregation

In [182]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 3:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416

In [183]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [184]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [185]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))

RMSE:  41.32360331281681
MAE:  24.15678114301703
R_Squared:  0.7488536194411535
